<a href="https://colab.research.google.com/github/abirhazra/Reinforcement-Learning/blob/main/DDPG_agent(Actor_Critic)_with_Keras_RL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install tensorflow==1.14
!pip install keras-rl

In [22]:
import numpy as np
import gym

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD, RMSprop

from rl.agents import  DDPGAgent
from rl.memory import SequentialMemory
from rl.random import  OrnsteinUhlenbeckProcess

In [23]:
ENV_NAME = 'Pendulum-v0'

In [24]:
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

[123]

In [25]:
nb_actions = env.action_space.shape[0]
nb_actions

1

In [26]:
actor = Sequential()
actor.add(Flatten(input_shape = (1,) + env.observation_space.shape))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('linear'))
print(actor.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                64        
_________________________________________________________________
activation_8 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_9 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_10 (Activation)   (None, 16)               

In [27]:
action_input = Input(shape=(nb_actions,),name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name= 'observation_input')
flattened_observation = Flatten()(observation_input)
x = Concatenate() ([action_input, flattened_observation])
x=Dense(32)(x)
x=Activation('relu')(x)
x=Dense(32)(x)
x=Activation('relu')(x)
x=Dense(32)(x)
x=Activation('relu')(x)
x=Dense(1)(x)
x=Activation('linear')(x)
critic = Model(inputs = [action_input, observation_input], outputs = x)
print(critic.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation_input (InputLayer)  [(None, 1, 3)]       0                                            
__________________________________________________________________________________________________
action_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 3)            0           observation_input[0][0]          
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 4)            0           action_input[0][0]               
                                                                 flatten_3[0][0]            

In [28]:
from rl.random import OrnsteinUhlenbeckProcess

In [29]:
memory = SequentialMemory(limit=100000, window_length = 1)
random_process = OrnsteinUhlenbeckProcess(size = nb_actions, theta = .15, mu = 0, sigma = 0.3)
agent = DDPGAgent(nb_actions = nb_actions, actor = actor, critic = critic, critic_action_input = action_input, memory = memory, nb_steps_warmup_critic = 100, nb_steps_warmup_actor = 100, random_process = random_process, gamma=.99, target_model_update = 1e-3 )

TypeError: ignored